In [1]:
import openai

In [3]:
import anthropic

In [16]:
from pydantic import BaseModel, Field
from typing import List
from anthropic import Anthropic
import os
import json

In [5]:
class TextAnalysis(BaseModel):
    sentiment: str = Field(description="Overall sentiment of the text (positive, negative, or neutral)")
    main_topics: List[str] = Field(description="List of main topics in the text")
    word_count: int = Field(description="Total word count of the text")

In [14]:
def analyze_text_with_claude(api_key: str, text: str) -> TextAnalysis:
    client = Anthropic(api_key=api_key)
 
    text_analysis_schema = TextAnalysis.model_json_schema()
 
    tools = [
        {
            "name": "build_text_analysis_result",
            "description": "build the text analysis object",
            "input_schema": text_analysis_schema
        }
    ]
 
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1200,
        temperature=0.2,
        system="You are analyzing the sentiment of a text.",
        messages=[
            {
                "role": "user",
                "content": f"{text}"
            }
        ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_text_analysis_result"}
    )
 
    function_call = message.content[0].input
    return TextAnalysis(**function_call)

# test it out!
api_key = os.environ.get("ANTHROPIC_API_KEY")
sample_text = "Claude tool use can be quite useful if used correctly."
 
print(f"Text: {sample_text}\nAnalyzing text...\n")
analysis = analyze_text_with_claude(api_key, sample_text)
print(f"Sentiment: {analysis.sentiment}")
print(f"Main topics: {', '.join(analysis.main_topics)}")
print(f"Word count: {analysis.word_count}")

Text: Claude tool use can be quite useful if used correctly.
Analyzing text...

Sentiment: positive
Main topics: tool use, Claude
Word count: 10


In [8]:
def list_anthropic_models(api_key: str) -> List[str]:
    client = Anthropic(api_key=api_key)
    models = client.models.list()
    return [model.id for model in models]  # Use id instead of name

# Test it out
models = list_anthropic_models(os.getenv("ANTHROPIC_API_KEY"))
print("Available Anthropic models:")
for model in models:
    print(f"- {model}")


Available Anthropic models:
- claude-3-5-sonnet-20241022
- claude-3-5-haiku-20241022
- claude-3-5-sonnet-20240620
- claude-3-haiku-20240307
- claude-3-opus-20240229
- claude-3-sonnet-20240229
- claude-2.1
- claude-2.0


In [22]:
# First, let's add our Pydantic models
class Step(BaseModel):
    action: str
    inputs: List[str]
    outputs: List[str]
    constraints: List[str] = Field(default_factory=list)

class ActionPattern(BaseModel):
    type: str
    frequency: str
    steps: List[Step]

class CurrentState(BaseModel):
    resources: dict = Field(default_factory=dict)
    constraints: dict = Field(default_factory=dict)
    progress_metrics: dict = Field(default_factory=dict)

class ExecutionStrategy(BaseModel):
    cycle: str
    checkpoints: List[str]
    memory_requirements: List[str]

class TaskPlan(BaseModel):
    goal: str
    success_metric: str
    current_state: CurrentState
    execution_strategy: ExecutionStrategy
    action_patterns: List[ActionPattern]

In [23]:
# Now let's fix the generate_task_plan function
def generate_task_plan(api_key: str, task_description: str) -> TaskPlan:
    client = Anthropic(api_key=api_key)
    
    task_plan_schema = TaskPlan.model_json_schema()
    
    tools = [
        {
            "name": "build_task_plan",
            "description": "Build a structured execution plan for any task with clear steps and requirements",
            "input_schema": task_plan_schema
        }
    ]

    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1500,
        temperature=0.2,
        system="You are a task planning expert who breaks down complex tasks into detailed, actionable steps.",
        messages=[
            {
                "role": "user",
                "content": f"Create a detailed execution plan for the following task: {task_description}"
            }
        ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_task_plan"}
    )

    return TaskPlan(**message.content[0].input)

In [24]:

task = "Research and compile healthy vegetarian dinner recipes for a week"
print(f"\nTask: {task}")
print("-" * 50)
plan = generate_task_plan(os.getenv("ANTHROPIC_API_KEY"), task)
print(json.dumps(plan.model_dump(), indent=2))


Task: Research and compile healthy vegetarian dinner recipes for a week
--------------------------------------------------
{
  "goal": "Research and compile healthy vegetarian dinner recipes for a week",
  "success_metric": "Have a set of 7 healthy vegetarian dinner recipes to cook for the week",
  "current_state": {
    "resources": {
      "time_available": "1 week",
      "cooking_experience": "intermediate"
    },
    "constraints": {
      "dietary_restrictions": "vegetarian"
    },
    "progress_metrics": {
      "number_of_recipes_found": 0
    }
  },
  "execution_strategy": {
    "cycle": "daily",
    "checkpoints": [
      "Identify recipe sources",
      "Compile list of potential recipes",
      "Evaluate recipes for health and feasibility",
      "Select 7 recipes"
    ],
    "memory_requirements": [
      "list of recipe sources",
      "list of potential recipes",
      "evaluation criteria for recipes"
    ]
  },
  "action_patterns": [
    {
      "type": "Research",
  

In this notebook, we're going to build an agentic agent that can do tasks by itself.

In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from standardbackend.tools.python_code_runner import tools as python_code_runner_tools
python_code_runner_tools

[Tool(name='run_python_code', description='Run Python code with configurable timeout and output limits', input_schema=<class 'standardbackend.tools.python_code_runner.EvalInput'>, execute=<function execute_python_code at 0x10cf436a0>)]

In [44]:
tools = python_code_runner_tools

# Convert to dict format when needed
tool_dicts = [tool.to_dict() for tool in tools]

In [45]:
from standardbackend.tools.python_code_runner import execute_python_code, EvalInput

# Test printing
print("Testing basic printing...")
r = execute_python_code(EvalInput(code="print('Hello world!')\nprint(f'Numbers: {1+1}')\nprint('Multiple\\nlines\\ntest')"))
print(r)

# Test directory listing
print("\nTesting directory listing...")
r = execute_python_code(EvalInput(code="""
import os
print('Current directory contents:')
print('\\n'.join(os.listdir('.')))
"""))
print(r)

# Test system RAM info
print("\nTesting system memory info...")
r = execute_python_code(EvalInput(code="""
import psutil
mem = psutil.virtual_memory()
print(f'Total RAM: {mem.total / (1024**3):.1f} GB')
print(f'Available RAM: {mem.available / (1024**3):.1f} GB') 
print(f'RAM Usage: {mem.percent}%')
"""))
print(r)

Testing basic printing...
Hello world!
Numbers: 2
Multiple
lines
test


Testing directory listing...
Current directory contents:
hello.ipynb


Testing system memory info...
Total RAM: 16.0 GB
Available RAM: 3.0 GB
RAM Usage: 81.5%



In [11]:
# Remember to call to_dict() on the tool to get the dictionary representation
tools[0].to_dict()

{'name': 'python_code_runner',
 'description': 'Run Python code with configurable timeout and output limits',
 'input_schema': {'properties': {'code': {'title': 'Code', 'type': 'string'},
   'timeout': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
    'default': 30,
    'title': 'Timeout'},
   'max_output_length': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
    'default': 1000,
    'title': 'Max Output Length'}},
  'required': ['code'],
  'title': 'EvalInput',
  'type': 'object'}}

How to use tools?

Extract tool input(s), run code, and return results: (API request)

On the client side, you should extract the tool name and input(s) from Claude's tool use request.
Run the actual tool code on the client side.
Return the results to Claude by continuing the conversation with a new user message containing a tool_result content block.


In [53]:
from anthropic import Anthropic
import os
from standardbackend.tools import ExecutionStatus, ToolCache
from standardbackend.tools.python_code_runner import tools as python_tools

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

tool_cache = ToolCache(python_tools)

def answer_question(question: str):
    messages = [
        {
            "role": "user", 
            "content": "How much memory do I have right now? Be sure to use the tool!"
        }
    ]
    
    claude_message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1200,
        temperature=0.2,
        tools=tool_cache.tool_specs,
        # tool_choice={"type": "tool", "name": "python_code_runner"},
        tool_choice={"type": "auto"},
        messages=messages
    )

    def parse_message(message):
        """Can return metadata, but for now we just build the response messages directly"""
        metadata = {
            'tools_called': []
        }
        tool_responses = []

        def handle_text_output(block):
            print(block.text)
        
        def handle_tool_use(block):
            print(f"[tool_use] {block.name} {block.id}")
            print("=> ", block.input)

            # Schedule execution - right now it executes immediately
            ans = tool_cache.request_execution(block.id, block.name, block.input)

            # Get the result!
            ans = tool_cache.get(block.id)

            if ans.status == ExecutionStatus.COMPLETED:
                tool_responses.append({
                    "role": "user",
                    "content": [
                        {
                        "type": "tool_result",
                        "tool_use_id": block.id,
                        "content": ans.result
                        }
                    ]
                })
            else:
                print(f"Tool {block.id} failed with error: {ans.error}")
            
            # sure, print it
            print(f"[tool_answer] {ans.result}")


        for block in message.content:
            content_type = block.type
            if content_type == "text":
                handle_text_output(block)
            elif content_type == "tool_use":
                handle_tool_use(block)
                metadata['tools_called'].append(block.id)
            else:
                raise ValueError(f"Unexpected message type: {content_type}")
        
        return metadata, tool_responses
    
    metadata, tool_responses = parse_message(claude_message)
    
    # add stuff to the conversation
    messages.append(claude_message)
    print(claude_message)
    messages.extend(tool_responses)

    return messages

messages = answer_question("How much memory do I have right now?")


Here is how we can check the amount of memory available in the current environment:
[tool_use] run_python_code toolu_01Q8sFHK3EnUoohV1vfzHNf6
=>  {'code': 'import psutil\n\ntotal_memory = psutil.virtual_memory().total\nprint(f"Total memory: {total_memory / (1024 ** 2):.2f} MB")'}
[tool_answer] Total memory: 16384.00 MB

Message(id='msg_018XHYaQRDBURUvsCTY5wyRd', content=[TextBlock(text='Here is how we can check the amount of memory available in the current environment:', type='text'), ToolUseBlock(id='toolu_01Q8sFHK3EnUoohV1vfzHNf6', input={'code': 'import psutil\n\ntotal_memory = psutil.virtual_memory().total\nprint(f"Total memory: {total_memory / (1024 ** 2):.2f} MB")'}, name='run_python_code', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=439, output_tokens=116))


Hm.. do we want to have threads be a composite of Message and json types?

```
Message(id='msg_018XHYaQRDBURUvsCTY5wyRd', content=[TextBlock(text='Here is how we can check the amount of memory available in the current environment:', type='text'), ToolUseBlock(id='toolu_01Q8sFHK3EnUoohV1vfzHNf6', input={'code': 'import psutil\n\ntotal_memory = psutil.virtual_memory().total\nprint(f"Total memory: {total_memory / (1024 ** 2):.2f} MB")'}, name='run_python_code', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=439, output_tokens=116))
```

In [54]:
def pretty_print_messages(messages):
    """Pretty print conversation messages with color coding"""
    from termcolor import colored
    
    for msg in messages:
        # Handle both dict and Message objects
        if isinstance(msg, dict):
            role = msg['role']
            content = msg['content']
        else:
            role = msg.role
            content = msg.content
        
        # Print role header
        if role == 'user':
            print(colored(f"\n[User]", 'green', attrs=['bold']))
        elif role == 'assistant':
            print(colored(f"\n[Assistant]", 'blue', attrs=['bold']))
        
        # Handle different content types
        if isinstance(content, str):
            # Simple text content
            print(content)
        elif isinstance(content, list):
            # Complex content with blocks
            for block in content:
                if isinstance(block, dict):
                    # Tool result block
                    if block['type'] == 'tool_result':
                        print(colored("\n[Tool Result]", 'yellow', attrs=['bold']))
                        print(block['content'])
                    # Message block from assistant
                    elif block['type'] == 'text':
                        print(block['text'])
                    elif block['type'] == 'tool_use':
                        print(colored("\n[Tool Use]", 'magenta', attrs=['bold']))
                        print(f"Tool: {block['name']}")
                        print(f"Input: {block['input']}")
                else:
                    # Handle typed blocks
                    if block.type == 'tool_result':
                        print(colored("\n[Tool Result]", 'yellow', attrs=['bold']))
                        print(block.content)
                    elif block.type == 'text':
                        print(block.text)
                    elif block.type == 'tool_use':
                        print(colored("\n[Tool Use]", 'magenta', attrs=['bold']))
                        print(f"Tool: {block.name}")
                        print(f"Input: {block.input}")

print("\nConversation:")
pretty_print_messages(messages)



Conversation:

[User]
How much memory do I have right now? Be sure to use the tool!

[Assistant]
Here is how we can check the amount of memory available in the current environment:

[Tool Use]
Tool: run_python_code
Input: {'code': 'import psutil\n\ntotal_memory = psutil.virtual_memory().total\nprint(f"Total memory: {total_memory / (1024 ** 2):.2f} MB")'}

[User]

[Tool Result]
Total memory: 16384.00 MB



In [55]:
from standardbackend.helpers.thread import Thread

# Basic usage - checking system memory
thread = Thread()
messages = thread.send_message("How much memory do I have right now?")
pretty_print_messages(messages)

I apologize, I do not have direct access to information about your computer's memory. As an AI assistant without direct access to your device, I do not have the ability to retrieve details about your system's hardware specifications. I can only provide information based on what you directly share with me or what is available in my general knowledge. If you would like to check the amount of memory on your computer, you would need to use tools or commands specific to your operating system. Let me know if there are any other ways I can try to assist you!

[User]
How much memory do I have right now?

[Assistant]
I apologize, I do not have direct access to information about your computer's memory. As an AI assistant without direct access to your device, I do not have the ability to retrieve details about your system's hardware specifications. I can only provide information based on what you directly share with me or what is available in my general knowledge. If you would like to check the a

In [56]:
# Using a different model with higher temperature for more creative responses
creative_thread = Thread(
    model="claude-3-opus-20240229",
    temperature=0.8,
    max_tokens=2000
)
messages = creative_thread.send_message("Write a short story about a programmer debugging code")
pretty_print_messages(messages)

<thinking>
The provided tool run_python_code seems relevant for writing a short story about a programmer debugging code. The required parameter is:

code (string): This would be the Python code to execute to generate the story. 

Since the user did not provide the actual code to run, I do not have enough information to call the tool directly. The user's request was to write a story, not to debug or run a specific piece of code. So I don't think I can reasonably infer the code parameter in this case.

To fully address the user's request, I will write the story directly in my response without using the run_python_code tool, since I have the ability to generate a story on my own without needing to execute Python code.
</thinking>

Here is a short story about a programmer debugging code:

Sarah stared at her computer screen, her brow furrowed in concentration. She had been working on this tricky piece of code for hours, but couldn't seem to pinpoint the bug that was causing it to crash. 



In [ ]:
# Chain of tool interactions
analysis_thread = Thread()
messages = analysis_thread.send_message("What's my current CPU usage?")
# Then ask for analysis of that data
messages = analysis_thread.send_message("Based on the CPU usage you just checked, would you recommend running a heavy computation task right now? Why?")
pretty_print_messages(messages)